In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import tensorflow as tf

# Read data and scale the same

In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]
X_train, X_valid, X_test = X_train / 255, X_valid / 255, X_test / 255

In [3]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

# Build Model with Selu

In [4]:
tf.keras.backend.clear_session()
tf.random.set_seed(124)


model_bn = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [5]:
model_bn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 batch_normalization (Batch  (None, 784)               3136      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 batch_normalization_1 (Bat  (None, 300)               1200      
 chNormalization)                                                
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_2 (Bat  (None, 100)               4

In [10]:
[(var.name, var.trainable) for var in model_bn.get_layer('batch_normalization').variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [12]:
model_bn.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              metrics=["accuracy"])

In [13]:
history = model_bn.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 6s 3ms/step - loss: 0.9240 - accuracy: 0.6908 - val_loss: 0.5963 - val_accuracy: 0.7918
Epoch 2/5
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6075 - accuracy: 0.7918 - val_loss: 0.5125 - val_accuracy: 0.8204
Epoch 3/5
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5420 - accuracy: 0.8127 - val_loss: 0.4728 - val_accuracy: 0.8290
Epoch 4/5
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5073 - accuracy: 0.8228 - val_loss: 0.4506 - val_accuracy: 0.8354
Epoch 5/5
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4809 - accuracy: 0.8319 - val_loss: 0.4333 - val_accuracy: 0.8416


# Put batch noram before activation function

In [14]:
tf.keras.backend.clear_session()
tf.random.set_seed(142)

model_bn1 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False), # removing bias (because batch noram has bias)
    tf.keras.layers.BatchNormalization(), # batch norm before activation
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

In [15]:
model_bn1.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              metrics=["accuracy"])

In [16]:
history = model_bn1.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 5s 3ms/step - loss: 1.1074 - accuracy: 0.6519 - val_loss: 0.7101 - val_accuracy: 0.7772
Epoch 2/5
1719/1719 [==============================] - 4s 3ms/step - loss: 0.7033 - accuracy: 0.7706 - val_loss: 0.5873 - val_accuracy: 0.8040
Epoch 3/5
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6163 - accuracy: 0.7961 - val_loss: 0.5308 - val_accuracy: 0.8166
Epoch 4/5
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5655 - accuracy: 0.8096 - val_loss: 0.4963 - val_accuracy: 0.8280
Epoch 5/5
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5351 - accuracy: 0.8176 - val_loss: 0.4718 - val_accuracy: 0.8314
